## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,54.72,87,25,4.76,light rain
1,1,Vao,NC,-22.6667,167.4833,74.21,64,100,23.80,overcast clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,59.18,83,95,1.99,overcast clouds
3,3,Syava,RU,58.0146,46.3222,44.22,90,96,6.53,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,44.26,70,75,6.91,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vao,NC,-22.6667,167.4833,74.21,64,100,23.80,overcast clouds
6,6,Faanui,PF,-16.4833,-151.7500,81.19,70,19,6.42,few clouds
8,8,Fatehgarh Churian,IN,31.8621,74.9553,80.46,21,7,7.09,clear sky
9,9,Kapaa,US,22.0752,-159.3190,76.12,82,27,5.01,scattered clouds
11,11,Siocon,PH,7.7061,122.1353,78.42,86,47,3.20,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
691,691,Veraval,IN,20.9000,70.3667,81.82,69,0,12.41,clear sky
694,694,Cabedelo,BR,-6.9811,-34.8339,83.07,76,92,15.26,overcast clouds
696,696,Bandarbeyla,SO,9.4942,50.8122,81.81,73,6,11.30,clear sky
697,697,Najran,SA,17.4924,44.1277,79.45,38,62,11.65,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vao,NC,74.21,overcast clouds,-22.6667,167.4833,
6,Faanui,PF,81.19,few clouds,-16.4833,-151.7500,
8,Fatehgarh Churian,IN,80.46,clear sky,31.8621,74.9553,
9,Kapaa,US,76.12,scattered clouds,22.0752,-159.3190,
11,Siocon,PH,78.42,scattered clouds,7.7061,122.1353,
13,Rikitea,PF,73.20,moderate rain,-23.1203,-134.9692,
14,Half Moon Bay,US,73.11,clear sky,37.4636,-122.4286,
16,Saint Andrews,US,85.59,scattered clouds,34.0454,-81.1293,
18,Batangafo,CF,80.92,overcast clouds,7.3008,18.2833,
23,Matara,LK,78.22,light rain,5.9485,80.5353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

hotel_list = []

# 6b. Iterate through the hotel DataFrame.

for i,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
       
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotels["results"][0]["name"]
    
        hotel_list.append(hotel_name)
        
    except:
        hotel_list.append(np.NaN)
        pass
               

In [9]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"] = hotel_list


clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name<dd>{City}</dd></dt>
<dt>Country Code<dd>{Country}</dd></dt>
<dt>Weather Description<dd>{Current Description}</dd></dt>
<dt>Maximum Temperature<dd>{Max Temp}</dd></dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [13]:
# 11a. Add a marker layer for each city to the map.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))